In [1]:
#@title
! pip install torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
! pip install torch-scatter==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-sparse==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-cluster==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-spline-conv==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-geometric

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 127.2MB 85kB/s 
     |████████████████████████████████| 5.4MB 45.0MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 8.2MB 5.5MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 14.3MB 3.7MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 12.1MB 776kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 4.8MB 4.2MB/s 
     |████████████████████████████████| 133kB 2.7MB/s 
     |██████████

In [0]:
! pip install torch
! pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
! pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 10.6MB 11.3MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 15.2MB 205kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 14.5MB 236kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 5.1MB 7.9MB/s 
     |████████████████████████████████| 133kB 5.0MB/s 
     |████████████████████████████████| 235kB 55.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=7a508a593375845551ef8d964d74c0986de04ca3afc9d6a0d21941728ff39e2b
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
Successfully built torch-geometric


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU
#sys.path.append('geometric')
import torch_geometric.transforms as T
#from torch_geometric.datasets import QM9
#import data_reader
from torch_geometric.nn import NNConv, Set2Set,GINConv, GCNConv, GraphConv,SAGEConv
from torch_geometric.data import DataLoader
from torch_geometric.utils import remove_self_loops
import torch_geometric.nn
#from ignite.contrib.metrics.regression._base import _BaseRegression


#@title
import os
import os.path as osp
import sys

import torch
import torch.nn.functional as F
from torch_sparse import coalesce
from torch_geometric.data import (InMemoryDataset, download_url, extract_zip,
                                  Data)
from torch.nn import BatchNorm1d
from torch.utils.data import Dataset
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import global_add_pool, global_mean_pool
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean


In [0]:
#@title


import pandas as pd
try:
    import rdkit # no rdkit in colab anyway
    from rdkit import Chem
    from rdkit import rdBase
    from rdkit.Chem.rdchem import HybridizationType
    from rdkit import RDConfig
    from rdkit.Chem import ChemicalFeatures
    from rdkit.Chem.rdchem import BondType as BT
    rdBase.DisableLog('rdApp.error')
except ImportError:
    rdkit = None



class QM9(InMemoryDataset):
  

    raw_url = ('https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/'
               'molnet_publish/qm9.zip')
    raw_url2 = 'https://ndownloader.figshare.com/files/3195404'
    processed_url = 'http://www.roemisch-drei.de/qm9.zip'

    if rdkit is not None:
        types = {'H': 0, 'C': 1, 'N': 2, 'O': 3, 'F': 4, 'Cl':5,'Br':6, 'I':7,'P':8, 'S':9}
        #add more Elements, numbering without specific rational
        bonds = {BT.SINGLE: 0, BT.DOUBLE: 1, BT.TRIPLE: 2, BT.AROMATIC: 3}

    def __init__(self, root, transform=None, pre_transform=None,
                 pre_filter=None):
        super(QM9, self).__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])# data is from processed

    @property
    def raw_file_names(self):
        if rdkit is None:
            return 'babel_data.pt'
        else:
            return ['mp_df_final_acidH.sdf', 'df_final_dGI.csv', 'uncharacterized.txt']

    @property
    def processed_file_names(self):
        return 'babel_data.pt'

    """def download(self):
        if rdkit is None:
            file_path = download_url(self.processed_url, self.raw_dir)
            extract_zip(file_path, self.raw_dir)
            os.unlink(file_path)
        else:
            file_path = download_url(self.raw_url, self.raw_dir)
            extract_zip(file_path, self.raw_dir)
            os.unlink(file_path)

            file_path = download_url(self.raw_url2, self.raw_dir)
            os.rename(
                osp.join(self.raw_dir, '3195404'),
                osp.join(self.raw_dir, 'uncharacterized.txt'))
"""
    def process(self):
        if rdkit is None:
            print('Using a pre-processed version of the dataset. Please '
                  'install `rdkit` to alternatively process the raw data.')

            self.data, self.slices = torch.load(self.raw_paths[0])
            data_list = [self.get(i) for i in range(len(self))]

            if self.pre_filter is not None:
                data_list = [d for d in data_list if self.pre_filter(d)]

            if self.pre_transform is not None:
                data_list = [self.pre_transform(d) for d in data_list]

            data, slices = self.collate(data_list)
            torch.save((data, slices), self.processed_paths[0])
            return

        with open(self.raw_paths[1], 'r') as f:#csv seems to be read
            target = f.read().split('\n')[:-1]
            target = [float(x) for x in target]#20 is the number of columns in the csv
            target = torch.tensor(target, dtype=torch.float)
            target = torch.cat([target[3:], target[:3]], dim=-1) #what happens here?
            target = target.view(target.size(0), -1)


        """with open(self.raw_paths[2], 'r') as f: # read me is read
            skip = [int(x.split()[0]) for x in f.read().split('\n')[9:-2]]
        assert len(skip) == 3054
"""
        suppl = Chem.SDMolSupplier(self.raw_paths[0], removeHs=False)#sdf is read
        fdef_name = osp.join(RDConfig.RDDataDir, 'BaseFeatures.fdef') #whats that
        factory = ChemicalFeatures.BuildFeatureFactory(fdef_name)

        data_list = []
        for i, mol in enumerate(suppl):
            if mol is None:
                continue
            """if i in skip:#something from the readme file is skipped???? looksa like it is capturing text
                continue"""

            text = suppl.GetItemText(i)#
            N = mol.GetNumAtoms()
                 
            pos = text.split('\n')[4:4 + N]
            pos = [[float(x) for x in line.split()[:3]] for line in pos]
            pos = torch.tensor(pos, dtype=torch.float)

            type_idx = []
            atomic_number = []
            acceptor = []
            donor = []
            aromatic = []
            sp = []
            sp2 = []
            sp3 = []
            num_hs = []
            for atom in mol.GetAtoms():
                type_idx.append(self.types[atom.GetSymbol()])
                atomic_number.append(atom.GetAtomicNum())
                donor.append(0)
                acceptor.append(0)
                aromatic.append(1 if atom.GetIsAromatic() else 0)
                hybridization = atom.GetHybridization()
                sp.append(1 if hybridization == HybridizationType.SP else 0)
                sp2.append(1 if hybridization == HybridizationType.SP2 else 0)
                sp3.append(1 if hybridization == HybridizationType.SP3 else 0)
                num_hs.append(atom.GetTotalNumHs(includeNeighbors=True))

            feats = factory.GetFeaturesForMol(mol)
            for j in range(0, len(feats)):
                if feats[j].GetFamily() == 'Donor':
                    node_list = feats[j].GetAtomIds()
                    for k in node_list:
                        donor[k] = 1
                elif feats[j].GetFamily() == 'Acceptor':
                    node_list = feats[j].GetAtomIds()
                    for k in node_list:
                        acceptor[k] = 1

            x1 = F.one_hot(torch.tensor(type_idx), num_classes=len(self.types))
            x2 = torch.tensor([
                atomic_number, acceptor, donor, aromatic, sp, sp2, sp3, num_hs
            ], dtype=torch.float).t().contiguous()
            x = torch.cat([x1.to(torch.float), x2], dim=-1)

            row, col, bond_idx = [], [], []
            for bond in mol.GetBonds():
                start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
                row += [start, end]
                col += [end, start]
                bond_idx += 2 * [self.bonds[bond.GetBondType()]]

            edge_index = torch.tensor([row, col], dtype=torch.long)
            edge_attr = F.one_hot(
                torch.tensor(bond_idx),
                num_classes=len(self.bonds)).to(torch.float)
            edge_index, edge_attr = coalesce(edge_index, edge_attr, N, N)

            y = target[i].unsqueeze(0) # gets safed as 1D list in Tensor insetad of just float
            
            name = mol.GetProp('_Name')

            data = Data(x=x, pos=pos, edge_index=edge_index,
                        edge_attr=edge_attr, y=y, name=name)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)

            data_list.append(data)
        
        torch.save(self.collate(data_list), self.processed_paths[0])

class MyTransform(object):
    def __call__(self, data):
        # Specify target.
        data.y = data.y[:, target]
        return data


class Complete(object):
    def __call__(self, data):
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data

target = 0
dim = 64

In [4]:
sys.path.append('/content/drive/My Drive/Master/Kaishi/geometric')
path = "drive/My Drive/Master/Kaishi/geometric/obabel_datasets"
#dataset=QM9(path)
transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
dataset = QM9(path, transform=transform).shuffle()
datasetI=dataset
dataset# if processing data exists the raw data wont be accessed anymore

QM9(5992)

In [5]:
class ChargeNet(torch.nn.Module):
  def __init__(self, conv=GCNConv, hidden_units=64):
    super(ChargeNet, self).__init__()
    self.conv1 = conv(dataset.num_features, hidden_units)
    self.conv2 = conv(hidden_units, hidden_units)
    self.conv3 = conv(hidden_units, 1)
    
  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    
    x1 = F.relu(self.conv1(x, edge_index))
    x2 = F.relu(self.conv2(x1, edge_index))
    return self.conv3(x2, edge_index)[:,0]
dataset.data.edge_index.shape

torch.Size([2, 402404])

In [0]:
debug=False
shuff=True
n_rnn=0
if debug:
  shuff=False
dataset = QM9(path, transform=transform).shuffle()

dim=64
norm=True
# Normalize targets to mean = 0 and std = 1.
if norm:
  mean = dataset.data.y.mean(dim=0, keepdim=True)
  std = dataset.data.y.std(dim=0, keepdim=True)
  dataset.data.y = (dataset.data.y - mean) / std
  mean, std = mean[:, target].item(), std[:, target].item()
  mean,std =float(mean),float(std)
  print("mean,std:",mean,std)
else:
  mean,std=0,1
  print("mean,std:",mean,std)
print("y0",dataset.data.y[0])

# Split datasets.
len_dt=len(dataset)
tr_dt=round(len_dt*0.8)
eval_dt=round(len_dt*0.1)
test_dataset = dataset[(tr_dt+eval_dt):]
val_dataset = dataset[tr_dt:(tr_dt+eval_dt)]
train_dataset = dataset[:tr_dt]#2760
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=shuff)

print("edge_indexe",dataset.data.edge_index.shape)
class NetI(torch.nn.Module):#rather fast .8 MAE
    def __init__(self):
        super(NetI, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, dim)
        self.conv2 = GCNConv(dim, dim)
        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2*dim, 1)#somehow 2 dims needed

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.set2set(x, data.batch)#pool
        #x = F.log_softmax(x, dim=1)
        x = F.relu(self.lin1(x))


        return x.view(-1)



class Net(torch.nn.Module):
    def __init__(self, conv=GCNConv, hidden_units=64):
        super(Net, self).__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)
        self.lin0_5 = torch.nn.Linear(dim, dim)
        self.lin0_5I = torch.nn.Linear(dim, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(128, dim * dim))
        self.conv0 = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)

        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin1_5 = torch.nn.Linear(dim, dim)
        #nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        #self.lin1_5I = NNConv(dim,dim,nn5)
        
        #self.conv5 = GINConv(nn5)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        out = F.relu(self.lin0(data.x))
        out = self.lin0_5(out)
        out = self.lin0_5I(out)
        h = out.unsqueeze(0)

        for i in range(n_rnn):
            #print("Data:",data,i,data.edge_attr.shape,"index",data.edge_index.shape)
            m = F.relu(self.conv0(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)#RNN
            out = out.squeeze(0)
            if debug:
              print("mmmm:",m.shape,"squeeze", out.shape)
              break
        out = self.set2set(out, data.batch)#pool
        #print(out.shape,"set2set out")
        out = F.relu(self.lin1(out))
        #print(out.shape,"l1 out")
        out = self.lin1_5(out)
        #out = F.relu(self.lin1_5I(out,edge_index, data.edge_attr))
        out = self.lin2(out)
        #out = self.conv1(out, data.edge_index)
        
        #out = self.conv3(out)
        return out.view(-1)

class NetII(torch.nn.Module):
    def __init__(self, conv=GraphConv, hidden_units=64):
        super(NetII, self).__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)
        self.lin0_5 = torch.nn.Linear(dim, 2*dim)
        self.lin0_5I = torch.nn.Linear(2*dim, 4*dim)
        self.lin0_5II = torch.nn.Linear(4*dim, 8*dim)
        self.lin0_5III = torch.nn.Linear(8*dim, 6*dim)
        self.lin0_5IV = torch.nn.Linear(6*dim, 2*dim)
        self.lin0_5V = torch.nn.Linear(2*dim, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(128, dim * dim))
        self.conv0 = NNConv(dim, dim, nn, aggr='mean')
        self.conv2 = torch_geometric.nn.GraphConv(dim, dim)
        #self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)

        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin1_5 = torch.nn.Linear(dim, dim)
        #self.conv2 = torch_geometric.nn.GCNConv(dim, dim)
        #self.conv2 = torch_geometric.nn.GCNConv(dim, dim)
        #nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        #self.lin1_5I = NNConv(dim,dim,nn5)
        
        #self.conv5 = GINConv(nn5)
        #self.set3set = Set2Set(dim, processing_steps=3)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        out = F.relu(self.lin0(data.x))
        out = self.lin0_5(out)
        out = self.lin0_5I(out)
        out = self.lin0_5II(out)
        out = self.lin0_5III(out)
        out = self.lin0_5IV(out)
        out = self.lin0_5V(out)
        h = out.unsqueeze(0)
        

        for i in range(n_rnn):
            #print("Data:",data,i,data.edge_attr.shape,"index",data.edge_index.shape)
            m = F.relu(self.conv0(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)#RNN
            out = out.squeeze(0)
            if debug:
              print("mmmm:",m.shape,"squeeze", out.shape)
              break
        out = F.relu(self.conv0(out, data.edge_index, data.edge_attr))
        out = self.conv2(out, edge_index)
        out = self.set2set(out, data.batch)#pool
        #print(out.shape,"set2set out")
        out = F.relu(self.lin1(out))
        #print(out.shape,"l1 out")
        out = self.lin1_5(out)
        #out = F.relu(self.lin1_5I(out,edge_index, data.edge_attr))
        #print("edge_indexe",data.edge_index.shape)
        
        #out = self.set3set(out, data.batch)#pool
        out = self.lin2(out)
        #out = self.conv1(out, data.edge_index)
        
        #out = self.conv3(out)
        return out.view(-1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model= Net(GCNConv, hidden_units=64)
model = NetII().to(device)
PATH="/content/drive/My Drive/Master/Kaishi/QM9_data/pretr_homo.pt"
#model.load_state_dict(torch.load(PATH))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.0000000001)


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        #print("x",data.x.dim(),"data",len(data.edge_attr), "IIIII",len(data.edge_index), data.y.shape)
        #break
        
        loss = F.mse_loss(model(data), data.y)
        if debug:
          #print(data)
          #print("Model:",model(data).shape)
          #print("Loss in MSE/num graphs",loss)
          None
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
        error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
    return error / len(loader.dataset)


best_val_error = None
n_ep=301
if debug:
  n_ep=2
for epoch in range(1, n_ep):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    val_error = test(val_loader)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(test_loader)
        best_val_error = val_error

    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}, '
          'Test MAE: {:.7f}'.format(epoch, lr, loss, val_error, test_error))
#PATH="/content/drive/My Drive/Master/Kaishi/QM9_data/pretr_homo.pt"
#torch.save(model.state_dict(), PATH)

mean,std: 15.908252716064453 5.60269832611084
y0 tensor([-0.2872])
edge_indexe torch.Size([2, 402404])
Epoch: 001, LR: 0.001000, Loss: 1.2999993, Validation MAE: 4.4098055, Test MAE: 4.5770816


In [0]:
#Convert normalized MAE to MAE in dG
xq,dx=0,0.38
dG=(xq*std)+mean
xqI=xq+dx
dGI=xqI*std+mean
print(abs(dG-dGI),dG)


In [0]:
mean = dataset.data.y.mean(dim=0, keepdim=True)
std = dataset.data.y.std(dim=0, keepdim=True)
dataset.data.y = (dataset.data.y - mean) / std
mean, std = mean[:, target].item(), std[:, target].item()
mean,std =float(mean),float(std)

In [0]:
mean,std=14.813546180725098,6.972548961639404

In [0]:
mean,std: 15.908252716064453 5.60269832611084

In [0]:
dataset.data.y[0]

NameError: ignored

In [0]:
Epoch: 300, LR: 0.000010, Loss: 37.9708740, Validation MAE: 5.1751591, Test MAE: 4.8465116 not normalized
Epoch: 300, LR: 0.000010, Loss: 0.3063120, Validation MAE: 0.4217039, Test MAE: 0.4503622
Epoch: 300, LR: 0.000010, Loss: 0.3584516, Validation MAE: 0.4840323, Test MAE: 0.4453856
Epoch: 300, LR: 0.000010, Loss: 0.4519423, Validation MAE: 0.4980229, Test MAE: 0.5080364
Epoch: 300, LR: 0.000010, Loss: 0.3482924, Validation MAE: 0.4792188, Test MAE: 0.4773444
Epoch: 300, LR: 0.000010, Loss: 38.9180153, Validation MAE: 4.7670103, Test MAE: 4.9181706 #not normalized
Epoch: 300, LR: 0.000010, Loss: 0.3711229, Validation MAE: 3.0032940, Test MAE: 3.2640099 normalized
Epoch: 300, LR: 0.000010, Loss: 0.2236194, Validation MAE: 2.3398583, Test MAE: 2.3467481 normalized new data

In [0]:
model = NetII().to(device)
PATH="/content/drive/My Drive/Master/Kaishi/QM9_data/pretr_homo.pt"

NameError: ignored

In [0]:
dataset.num_features

18